In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import chess
import  numpy as np
import board_util

2024-02-23 17:04:22.842368: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-23 17:04:22.888840: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 17:04:22.888867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 17:04:22.890003: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-23 17:04:22.895900: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-23 17:04:22.896744: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# Read in data
file_path = "dataset_fen.txt"
data = []
with open(file_path, "r") as file:
    lines = file.readlines()
    print(len(lines ))
for line in lines:
    chessboard_state, move = line.split("[MOVESEP]")
    cb = chess.Board(chessboard_state)
    bb = board_util.board_to_bitboards(cb)
    ab = board_util.bitboards_to_array(bb)
   
    encode_mobe = board_util.encode_move(move.strip(), ab)
    data.append((ab, encode_mobe))
print(data[0])
print(len(data))
    



391206
(array([[[0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 1, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 1, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[1, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],


In [ ]:
print(len(data))

In [19]:
import board_util
cb = chess.Board('rnbqkbnr/pp1ppppp/8/2p5/P7/8/1PPPPPPP/RNBQKBNR w KQkq - 0 2')
bb = board_util.board_to_bitboards(cb)
ab = board_util.bitboards_to_array(bb)

In [20]:

moves = list(cb.legal_moves)
move = moves[0]
uci = move.uci()
print(uci)
encode = board_util.encode_move(uci, ab)
# for move in cb.legal_moves:
#     uci = move.uci()
#     bits = board_util.encode_move(uci, ab)
#     print(uci, bits)
print("encoded move:", encode)

g1h3
piece channel [[0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0]]
encoded move: [0 6 2 7 0]


In [8]:
num_squares = 64
output_units_start_square = num_squares
output_units_end_square = num_squares
## Cnn model.
model = models.Sequential([
    tf.keras.Input(shape=(8, 8, 17)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),

    # Output layer
    tf.keras.layers.Dense(output_units_start_square, activation='softmax', name='output_start_square'),
    tf.keras.layers.Dense(output_units_end_square, activation='softmax', name='output_end_square'),
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
